# DES Y6 Deep Field Exposures:  u-band Photometric Zeropoints tied to GALEX + GaiaEDR3

## 1. Setup

In [29]:
import numpy as np
import pandas as pd
from scipy import interpolate
import glob
import math
import os
import subprocess
import sys
import gc

import glob
import pickle

import easyaccess as ea

#import AlasBabylon

import fitsio
from astropy.io import fits
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, vstack

import tempfile

import matplotlib.pyplot as plt
 
%matplotlib inline

In [30]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## 2. User Input

### 2.1.  General User Input  

In [31]:
verbose = 1
tag_des = 'Y6A2_FINALCUT'      # Official tag for DES Y6A2_FINALCUT
tag_decade = 'DECADE_FINALCUT' # Tag for DECADE
rawdata_dir = '../RawData'
zeropoints_dir='../Zeropoints'

bandList = ['u']

### 2.2. Logical Variables to Indicate which Code Cells to Run

In [32]:
do_calc_galex_gaiaedr3_zps = True

### 2.3. Sky Region Definitions

In [33]:
region_name_list = [
                    'VVDSF14', 
                    'VVDSF22', 
                    'DEEP2', 
                    'SN-E', 
                    'SN-X_err', 
                    'SN-X', 
                    'ALHAMBRA2', 
                    'SN-S',
                    'SN-C', 
                    'EDFS',
                    'MACS0416',
                    'SN-S_err',
                    'COSMOS'
                    ]

region_ramin = {
                'VVDSF14':208.,
                'VVDSF22':333.,
                'DEEP2':351., 
                'SN-E':6., 
                'SN-X_err':13., 
                'SN-X':32., 
                'ALHAMBRA2':35., 
                'SN-S':39.5,
                'SN-C':50., 
                'EDFS':55.,
                'MACS0416':62.,
                'SN-S_err':83.5,
                'COSMOS':148.
               }

region_ramax = {
                'VVDSF14':212.,
                'VVDSF22':337.,
                'DEEP2':354., 
                'SN-E':12., 
                'SN-X_err':17., 
                'SN-X':38., 
                'ALHAMBRA2':39., 
                'SN-S':44.5,
                'SN-C':56., 
                'EDFS':67.,
                'MACS0416':66.,
                'SN-S_err':88.,
                'COSMOS':153.
               }

region_decmin = {
                'VVDSF14':3.,
                'VVDSF22':-1.5,
                'DEEP2':-1.5, 
                'SN-E':-46., 
                'SN-X_err':-32., 
                'SN-X':-8., 
                'ALHAMBRA2':-0.5, 
                'SN-S':-2.5,
                'SN-C':-31., 
                'EDFS':-51.,
                'MACS0416':-25.5,
                'SN-S_err':-38.,
                'COSMOS':0.5
               }


region_decmax = {
                'VVDSF14':7.,
                'VVDSF22':1.5,
                'DEEP2':1.5, 
                'SN-E':-41., 
                'SN-X_err':-29., 
                'SN-X':-3., 
                'ALHAMBRA2':2.5, 
                'SN-S':1.5,
                'SN-C':-25., 
                'EDFS':-46.,
                'MACS0416':-22.5,
                'SN-S_err':-35.,
                'COSMOS':4.0
               }


for regionName in region_name_list:
    print regionName, region_ramin[regionName], region_ramax[regionName], region_decmin[regionName], region_decmax[regionName]

VVDSF14 208.0 212.0 3.0 7.0
VVDSF22 333.0 337.0 -1.5 1.5
DEEP2 351.0 354.0 -1.5 1.5
SN-E 6.0 12.0 -46.0 -41.0
SN-X_err 13.0 17.0 -32.0 -29.0
SN-X 32.0 38.0 -8.0 -3.0
ALHAMBRA2 35.0 39.0 -0.5 2.5
SN-S 39.5 44.5 -2.5 1.5
SN-C 50.0 56.0 -31.0 -25.0
EDFS 55.0 67.0 -51.0 -46.0
MACS0416 62.0 66.0 -25.5 -22.5
SN-S_err 83.5 88.0 -38.0 -35.0
COSMOS 148.0 153.0 0.5 4.0


### 2.4. Check on Location of TMPDIR...

In [34]:
# Check on TMPDIR... 
tempfile.gettempdir()

'/data/des40.a/data/dtucker/TmpDir'

In [35]:
# Set tmpdir variable to $TMPDIR (for future reference)...
tmpdir = os.environ['TMPDIR']

### 2.5. Create Main Zeropoints Directory (if it does not already exist)...

In [36]:
# Create main Zeropoints directory, if it does not already exist...
if not os.path.exists(zeropoints_dir):
    os.makedirs(zeropoints_dir)

## 3. Useful Modules

In [37]:
def DECam_tie_to_galex_gaiaedr3(inputFile, outputFile, band, fluxObsColName, fluxerrObsColName, aggFieldColName, verbose):

    import numpy as np 
    import os
    import sys
    import datetime
    import pandas as pd
    from astropy.table import Table, vstack
    
    validBandsList = ['u']

    if band not in validBandsList:
        print """Filter band %s is not currently handled...  Exiting now!""" % (band)
        return 1
    
    reqColList = ['NUVmag', 'e_NUVmag',
                  'phot_g_mean_mag_corrected', 'phot_g_mean_mag_error_corrected',
                  'bp_rp','phot_bp_mean_mag_error', 'phot_rp_mean_mag_error', 
                  'GLAT', 'GLON', 'E(B-V)', 
                  fluxObsColName,fluxerrObsColName,aggFieldColName]        
      
    # Does the input file exist?
    if os.path.isfile(inputFile)==False:
        print """DECam_tie_to_galex_gaiaedr3 input file %s does not exist.  Exiting...""" % (inputFile)
        return 1

    # Read inputFile into a pandas DataFrame...
    print datetime.datetime.now()
    print """Reading in %s as a pandas DataFrame...""" % (inputFile)
    t = Table.read(inputFile)
    dataFrame = t.to_pandas()
    print datetime.datetime.now()
    print

    reqColFlag = 0
    colList = dataFrame.columns.tolist()
    for reqCol in reqColList:
        if reqCol not in colList:
            print """ERROR:  Required column %s is not in the header""" % (reqCol)
            reqColFlag = 1
    if reqColFlag == 1:
        print """Missing required columns in header of %s...  Exiting now!""" (inputFile)
        return 1

    # Identify column of the standard magnitude for the given band...
    magStdColName = """MAG_STD_%s""" % (band.upper())
    magerrStdColName = """MAGERR_STD_%s""" % (band.upper())
    
    # Rename columns (for convenience)...
    dataFrame.rename(columns={
                   'NUVmag':'NUV_galex',
                   'FUVmag':'FUV_galex',
                   'e_NUVmag':'NUVerr_galex',
                   'e_FUVmag':'FUVerr_galex',
                   'E(B-V)':'E_BV',                   
                   'phot_g_mean_mag_corrected':'G_gaia',
                   'phot_g_mean_mag_error_corrected':'G_err_gaia',
                   'bp_rp':'BP_RP_gaia',
                   'phot_bp_mean_mag_error':'BP_err_gaia',
                   'phot_rp_mean_mag_error':'RP_err_gaia',
                  },inplace=True)
   
    # Transform GALEX + GaiaEDR3 mags to DES mags for the given band...
    dataFrame = transGALEXGaiaEDR3toDES(dataFrame, band, magStdColName, magerrStdColName)

    # Add a 'MAG_OBS' column and a 'MAG_DIFF' column to the pandas DataFrame...
    dataFrame['MAG_OBS'] = -2.5*np.log10(dataFrame[fluxObsColName])
    dataFrame['MAG_DIFF'] = dataFrame[magStdColName]-dataFrame['MAG_OBS']


    ###############################################
    # Aggregate by aggFieldColName
    ###############################################

    # Make a copy of original dataFrame...
    df = dataFrame.copy()

    # Create an initial mask...
    mask1 = ( (df[magStdColName] >= 0.) & (df[magStdColName] <= 25.) )
    mask1 = ( mask1 & (df[fluxObsColName] > 10.) & (df['FLAGS'] < 2) & (np.abs(df['SPREAD_MODEL']) < 0.01))
    if magerrStdColName != 'None':  
        mask1 = ( mask1 & (df[magerrStdColName] < 0.1) )
    magDiffGlobalMedian = df[mask1]['MAG_DIFF'].median()
    magDiffMin = magDiffGlobalMedian - 5.0
    magDiffMax = magDiffGlobalMedian + 5.0
    mask2 = ( (df['MAG_DIFF'] > magDiffMin) & (df['MAG_DIFF'] < magDiffMax) )
    mask = mask1 & mask2

    # Iterate over the copy of dataFrame 3 times, removing outliers...
    #  We are using "Method 2/Group by item" from
    #  http://nbviewer.jupyter.org/urls/bitbucket.org/hrojas/learn-pandas/raw/master/lessons/07%20-%20Lesson.ipynb
    print "Sigma-clipping..."
    niter = 0
    for i in range(3):

        niter = i + 1
        print """   iter%d...""" % ( niter )

        # make a copy of original df, and then delete the old one...
        newdf = df[mask].copy()
        del df
        # group by aggFieldColName...
        grpnewdf = newdf.groupby([aggFieldColName])
        
        # add/update new columns to newdf
        print datetime.datetime.now()
        newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 3.00*x.std() )
        #newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 2.00*x.std() )
        print datetime.datetime.now()
        del grpnewdf
        print datetime.datetime.now()
        #print newdf

        nrows = newdf['MAG_DIFF'].size
        print """  Number of rows remaining:  %d""" % ( nrows )

        df = newdf
        mask = ( df['Outlier'] == False )  


    # Perform pandas grouping/aggregating functions on sigma-clipped Data Frame...
    print datetime.datetime.now()
    print 'Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...'
    groupedDataFrame = df.groupby([aggFieldColName])
    magZeroMedian = groupedDataFrame['MAG_DIFF'].median()
    magZeroMean = groupedDataFrame['MAG_DIFF'].mean()
    magZeroStd = groupedDataFrame['MAG_DIFF'].std()
    magZeroNum = groupedDataFrame['MAG_DIFF'].count()
    magZeroErr = magZeroStd/np.sqrt(magZeroNum-1)
    print datetime.datetime.now()
    print

    # Rename these pandas series...
    magZeroMedian.name = 'MAG_ZERO_MEDIAN'
    magZeroMean.name = 'MAG_ZERO_MEAN'
    magZeroStd.name = 'MAG_ZERO_STD'
    magZeroNum.name = 'MAG_ZERO_NUM'
    magZeroErr.name = 'MAG_ZERO_MEAN_ERR'

    # Also, calculate group medians for all columns in df that have a numerical data type...
    numericalColList = df.select_dtypes(include=[np.number]).columns.tolist()
    groupedDataMedian = {}
    for numericalCol in numericalColList:
        groupedDataMedian[numericalCol] = groupedDataFrame[numericalCol].median()
        groupedDataMedian[numericalCol].name = """%s_MEDIAN""" % (numericalCol)

    # Create new data frame containing all the relevant aggregate quantities
    #newDataFrame = pd.concat( [magZeroMedian, magZeroMean, magZeroStd, \
    #                           magZeroErr, magZeroNum], \
    #                           join='outer', axis=1 )
    seriesList = []
    for numericalCol in numericalColList:
        seriesList.append(groupedDataMedian[numericalCol])
    seriesList.extend([magZeroMedian, magZeroMean, magZeroStd, \
                               magZeroErr, magZeroNum])
    #print seriesList
    newDataFrame = pd.concat( seriesList, join='outer', axis=1 )
                               

    #newDataFrame.index.rename('FILENAME', inplace=True)

    # Saving catname-based results to output files...
    print datetime.datetime.now()
    print """Writing %s output file (using pandas to_csv method)...""" % (outputFile)
    newDataFrame.to_csv(outputFile, float_format='%.4f')
    print datetime.datetime.now()
    print

    return 0

In [38]:
# Transform GALEX+GaiaEDR3 mags into DES mags for this filter band...
def transGALEXGaiaEDR3toDES(dataFrame, band, magStdColName, magerrStdColName):

    import numpy as np 
    import pandas as pd
    
    if band is 'u':
        # u_sdss = 1.355 - 0.919*(NUV-G) + 0.010*(NUV-G)**2 
        #                - 1.780*(BP-RP) + 1.637*(BP-RP)**2
        #                - 0.0005*abs(GLAT) 
        dataFrame['u_sdss'] = dataFrame['NUV_galex'] + 1.355 \
                              - 0.919*(dataFrame['NUV_galex']-dataFrame['G_gaia']) \
                              + 0.010*(dataFrame['NUV_galex']-dataFrame['G_gaia'])*(dataFrame['NUV_galex']-dataFrame['G_gaia']) \
                              - 1.780*dataFrame['BP_RP_gaia'] \
                              + 1.637*dataFrame['BP_RP_gaia']*dataFrame['BP_RP_gaia'] \
                              - 0.0005*np.abs(dataFrame['GLAT'])
            

        dataFrame['gr_sdss'] = dataFrame['BP_RP_gaia'] - 0.165*dataFrame['BP_RP_gaia'] - 0.270
        
        dataFrame['u_des'] = dataFrame['u_sdss'] - 0.479 \
                                + 0.466*dataFrame['gr_sdss'] \
                                - 0.350*dataFrame['gr_sdss']*dataFrame['gr_sdss']

            
        dataFrame[magStdColName] = dataFrame['u_des']
        dataFrame[magerrStdColName] = dataFrame['G_err_gaia']  # temporary
        mask  = ( (dataFrame['NUV_galex']-dataFrame['G_gaia']) > +2.0)
        mask &= ( (dataFrame['NUV_galex']-dataFrame['G_gaia']) <= 7.5)
        mask &= ( (dataFrame['BP_RP_gaia']) >  0.6)
        mask &= ( (dataFrame['BP_RP_gaia']) <= 1.0)


    else:
        msg = "Unrecognized band: %s "%band
        raise ValueError(msg)

    dataFrame = dataFrame[mask].copy()

    return dataFrame

## 4. Zeropoints by tying to DES-transformed GALEX + Gaia EDR3 Stars

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

Calculate zeropoints region by region...

In [39]:
%%time 

if do_calc_galex_gaiaedr3_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.galex.gaiaedr3.fits""" % (subdir, regionName, band)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    

                    status = DECam_tie_to_galex_gaiaedr3(inputFile, outputFile, 
                                                    band, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.?.u.galex.gaiaedr3.fits

# # # # # # # # # # # # # # #
Working on region VVDSF22
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF22.?.u.galex.gaiaedr3.fits

# # # # # # # # # # # # # # #
Working on region DEEP2
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.DEEP2.?.u.galex.gaiaedr3.fits

# # # # # # # # # # # # # # #
Working on region SN-E
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.u.galex.gaiaedr3.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.u.galex.gaiaedr3.csv
2022-02-04 18:57:14.886613
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.u.galex.gaiaedr3.fits as a pan

2022-02-04 18:58:29.202592

Sigma-clipping...
   iter1...
2022-02-04 18:58:29.847564
2022-02-04 18:58:32.207255
2022-02-04 18:58:32.208222
  Number of rows remaining:  32506
   iter2...
2022-02-04 18:58:32.409320
2022-02-04 18:58:34.769425
2022-02-04 18:58:34.770465
  Number of rows remaining:  32268
   iter3...
2022-02-04 18:58:34.906132
2022-02-04 18:58:37.262771
2022-02-04 18:58:37.263821
  Number of rows remaining:  32252
2022-02-04 18:58:37.264906
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-02-04 18:58:37.278982

2022-02-04 18:58:37.688812
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.u.galex.gaiaedr3.csv output file (using pandas to_csv method)...
2022-02-04 18:58:39.021972


CPU times: user 1min 19s, sys: 884 ms, total: 1min 19s
Wall time: 1min 24s


In [40]:
inputFile = '../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.MACS0416.0.u.galex.gaiaedr3.fits'
t = Table.read(inputFile)
dataFrame = t.to_pandas()

In [41]:
dataFrame

,EXPNUM,EXPRA,EXPDEC,EXPTIME,AIRMASS,BAND,NITE,MJD_OBS,FIELD,OBJECT,...,phot_g_mean_mag_corrected,phot_g_mean_mag_error_corrected,phot_g_mean_flux_corrected,phot_bp_rp_excess_factor_corrected,ra_epoch2000,dec_epoch2000,ra_epoch2000_error,dec_epoch2000_error,ra_dec_epoch2000_corr,angDist_cds
0,807637.0,64.091671,-24.016400,360.0,1.01,u,20181230.0,58483.123162,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.006886
1,807635.0,64.125000,-24.041667,360.0,1.01,u,20181230.0,58483.113609,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.016071
2,807034.0,64.091671,-24.016400,360.0,1.02,u,20181228.0,58481.139452,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.013475
3,807323.0,64.041658,-24.029034,360.0,1.02,u,20181229.0,58482.130174,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.017292
4,807321.0,64.058342,-23.965870,360.0,1.01,u,20181229.0,58482.120595,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.049662
5,807639.0,64.108333,-23.978503,360.0,1.02,u,20181230.0,58483.132754,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.101149
6,807032.0,64.125000,-24.041667,360.0,1.02,u,20181228.0,58481.129849,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.057934
7,807037.0,64.024996,-24.092200,360.0,1.04,u,20181228.0,58481.153792,N,MACS J0416.1-2403,...,12.521453,0.002760,184657.0000,-0.007,64.798382,-24.031911,0.1278,0.1988,-0.4634,0.019348
8,807327.0,64.158329,-24.066934,360.0,1.04,u,20181229.0,58482.149335,N,MACS J0416.1-2403,...,12.521453,0.002760,184657.0000,-0.007,64.798382,-24.031911,0.1278,0.1988,-0.4634,0.012220
9,807324.0,64.075004,-24.079567,360.0,1.02,u,20181229.0,58482.134979,N,MACS J0416.1-2403,...,12.521453,0.002760,184657.0000,-0.007,64.798382,-24.031911,0.1278,0.1988,-0.4634,0.013560


In [42]:
magStdColName = """MAG_STD_%s""" % (band.upper())
magerrStdColName = """MAGERR_STD_%s""" % (band.upper())
print magStdColName
print magerrStdColName

MAG_STD_U
MAGERR_STD_U


In [43]:
# Rename columns (for convenience)...
dataFrame.rename(columns={
                   'NUVmag':'NUV_galex',
                   'FUVmag':'FUV_galex',
                   'e_NUVmag':'NUVerr_galex',
                   'e_FUVmag':'FUVerr_galex',
                   'E(B-V)':'E_BV',                   
                   'phot_g_mean_mag_corrected':'G_gaia',
                   'phot_g_mean_mag_error_corrected':'G_err_gaia',
                   'bp_rp':'BP_RP_gaia',
                   'phot_bp_mean_mag_error':'BP_err_gaia',
                   'phot_rp_mean_mag_error':'RP_err_gaia',
                  },inplace=True)

dataFrame

,EXPNUM,EXPRA,EXPDEC,EXPTIME,AIRMASS,BAND,NITE,MJD_OBS,FIELD,OBJECT,...,G_gaia,G_err_gaia,phot_g_mean_flux_corrected,phot_bp_rp_excess_factor_corrected,ra_epoch2000,dec_epoch2000,ra_epoch2000_error,dec_epoch2000_error,ra_dec_epoch2000_corr,angDist_cds
0,807637.0,64.091671,-24.016400,360.0,1.01,u,20181230.0,58483.123162,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.006886
1,807635.0,64.125000,-24.041667,360.0,1.01,u,20181230.0,58483.113609,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.016071
2,807034.0,64.091671,-24.016400,360.0,1.02,u,20181228.0,58481.139452,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.013475
3,807323.0,64.041658,-24.029034,360.0,1.02,u,20181229.0,58482.130174,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.017292
4,807321.0,64.058342,-23.965870,360.0,1.01,u,20181229.0,58482.120595,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.049662
5,807639.0,64.108333,-23.978503,360.0,1.02,u,20181230.0,58483.132754,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.101149
6,807032.0,64.125000,-24.041667,360.0,1.02,u,20181228.0,58481.129849,N,MACS J0416.1-2403,...,21.872002,0.044421,33.5851,6.789,64.797476,-23.920509,24.0401,22.1076,-0.8619,0.057934
7,807037.0,64.024996,-24.092200,360.0,1.04,u,20181228.0,58481.153792,N,MACS J0416.1-2403,...,12.521453,0.002760,184657.0000,-0.007,64.798382,-24.031911,0.1278,0.1988,-0.4634,0.019348
8,807327.0,64.158329,-24.066934,360.0,1.04,u,20181229.0,58482.149335,N,MACS J0416.1-2403,...,12.521453,0.002760,184657.0000,-0.007,64.798382,-24.031911,0.1278,0.1988,-0.4634,0.012220
9,807324.0,64.075004,-24.079567,360.0,1.02,u,20181229.0,58482.134979,N,MACS J0416.1-2403,...,12.521453,0.002760,184657.0000,-0.007,64.798382,-24.031911,0.1278,0.1988,-0.4634,0.013560


In [44]:
dataFrame = transGALEXGaiaEDR3toDES(dataFrame, band, magStdColName, magerrStdColName)
dataFrame[['G_gaia', 'MAG_STD_U']]

,G_gaia,MAG_STD_U
7,12.521453,13.907427
8,12.521453,13.907427
9,12.521453,13.907427
10,12.521453,13.907427
11,12.521453,13.907427
12,12.521453,13.907427
13,12.521453,13.907427
14,12.521453,13.907427
15,12.521453,13.907427
16,18.302755,19.158591


Combine region-by-region results into a single file...

In [45]:
%%time 

if do_calc_galex_gaiaedr3_zps: 
        
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.galex.gaiaedr3.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.galex.gaiaedr3.csv""" % (subdir, band)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.galex.gaiaedr3.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band u
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.u.galex.gaiaedr3.csv
CPU times: user 4.02 s, sys: 0 ns, total: 4.02 s
Wall time: 4.11 s


**DECADE:**

Calculate zeropoints region by region...

In [46]:
%%time 

if do_calc_galex_gaiaedr3_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.galex.gaiaedr3.fits""" % (subdir, regionName, band)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    

                    status = DECam_tie_to_galex_gaiaedr3(inputFile, outputFile, 
                                                    band, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.u.galex.gaiaedr3.fits

# # # # # # # # # # # # # # #
Working on region VVDSF22
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.?.u.galex.gaiaedr3.fits

# # # # # # # # # # # # # # #
Working on region DEEP2
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.DEEP2.?.u.galex.gaiaedr3.fits

# # # # # # # # # # # # # # #
Working on region SN-E
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-E.0.u.galex.gaiaedr3.fits
../Zeropoints/zps_DECADE_FINALCUT.SN-E.0.u.galex.gaiaedr3.csv
2022-02-04 18:59:52.125707
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-E.0.u.galex.gaiaedr3.fits as a pandas DataFrame...
2022-

Combine region-by-region results into a single file...

In [47]:
%%time 

if do_calc_galex_gaiaedr3_zps: 
        
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.galex.gaiaedr3.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.galex.gaiaedr3.csv""" % (subdir, band)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.galex.gaiaedr3.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band u
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.u.galex.gaiaedr3.csv
CPU times: user 667 ms, sys: 0 ns, total: 667 ms
Wall time: 661 ms
